In [1]:
import numpy as np
import numpy.random as npr
from IPython.display import Latex

In [2]:
npr.seed(10)

sites = np.ones(12)*1/2*np.sign(npr.random(12)-0.5)
links = np.ones(12)*1/2*np.sign(npr.random(12)-0.5)
print(sites)
print(links)

[ 0.5 -0.5  0.5  0.5 -0.5 -0.5 -0.5  0.5 -0.5 -0.5  0.5  0.5]
[-0.5  0.5  0.5  0.5  0.5 -0.5  0.5  0.5  0.5 -0.5 -0.5  0.5]


In [3]:
leg_sites = {
    1/2 : '$\circ$',
    -1/2 : '$\\bullet$'
}

leg_links = {
    1/2 : '$\leftarrow$',
    -1/2 : '$\\rightarrow$'
}

def Visualize(sites, links):
    sites_symbols = np.vectorize(leg_sites.get)(sites)
    links_symbols = np.vectorize(leg_links.get)(links)
    seq_list = [i + j for i, j in zip(sites_symbols, links_symbols)]
    total_list = ''.join(seq_list)
    return total_list

def Gaggiublock(i, sites, links):
    s1, l1 = sites[i],links[i]
    s2, l2 = sites[i+1],links[i+1]
    g = s2 - l2 + l1 + (-1)**(i)*1/2 == 0     
    '''
    Since numering start from 0 instead of 1 like in the review, 
    we absorbe a '-1' sign (exponent of last term becomes i instead of i+1)
    '''
    return g

In [4]:
sigma1 = (-1/2,-1/2)
sigma2 = (1/2,1/2)

sites = sigma1[0], sigma2[0]
links = sigma1[1], sigma2[1]
    
print(sites, links)
    
Gaggiublock(0,sites,links)

(-0.5, 0.5) (-0.5, 0.5)


True

In [5]:
from itertools import permutations, product

all_sit = list(product([0.5,-0.5], repeat=12))
all_lin = list(product([0.5,-0.5], repeat=12))
ok = []

for s in all_sit:
    for l in all_lin:
        check = True
        for i in range(len(s)-1):
            if not Gaggiublock(i, s, l): 
                check = False
                break
        if check == True: ok.append(np.array([s,l]))

In [6]:
n = 10
x = Visualize(ok[n][0],ok[n][1]) 
Latex(x)

<IPython.core.display.Latex object>

In [7]:
len(ok)

754

In [8]:
sequences_flattened=[]
for i in range(len(ok)):
    sequences_flattened.append(ok[i].flatten())

# Definizione della RBM


In [12]:
def activate(v_in, wei, bias, DE, info=False):
    act = np.dot(v_in, wei) + bias
    prob = 1. / (1. + np.exp(-DE*act))
    n = len(act)
    v_out = np.full(n, vmin)
    v_out[np.random.random_sample(n) < prob] = 1
    if info:
        print('input=', v_in)
        print('act=', act)
        print('prob=', prob)
        print('out=', v_out)
        
    return(v_out)

def training_RBM(alpha, w, a, b, seq):
    
    number_sequences = len(seq)
    sequence_length = len(seq[0])
    hidden_units = alpha*sequence_length #setting nubmer of hidden units
    
    #minibatches
    mini, m = int(0.08*number_sequences), 0 
    #epochs
    nepoch = 50
    #l_rate
    l_rate = 1.

    for epoch in range(1, 1+nepoch):
        for n in range(number_sequences):
            if m==0:
                #initialize
                v_data, v_model = np.zeros(sequence_length), np.zeros(sequence_length) 
                h_data, h_model = np.zeros(hidden_units), np.zeros(hidden_units) 
                vh_data, vh_model = np.zeros((sequence_length,hidden_units)), np.zeros((sequence_length, hidden_units)) 

            #CAPIRE BENE IL GAP
            #positive CD phase
            h = activate(v[n],w,b,GAP)
            #negative CD phase
            vf = activate(h, w.T,a,GAP)
            #positive CD nr 2
            hf = activate(vf,w,b, GAP)

            v_data += v[n]
            v_model += vf
            h_data += h
            h_model += hf
            
            theta = np.dot(w.T,seq.T)+b
            
            O_a += seq.T
            O_b += np.tanh(theta)
            O_w += 
            vh_model += np.outer(vf.T, hf)

            m += 1
            if m==mini:
                C = l_rate / mini
                
                dw = C*(vh_data - vh_model)
                da = C*(v_data - v_model)
                db = C*(h_data - h_model)

                w += dw
                a += da
                b += db
                m = 0

        #randomize order
        np.random.shuffle(v)
        l_rate = l_rate / (0.05 * l_rate + 1)
        
    return w,a,b

In [ ]:
def training_RBM(alpha, w, a, b, seq):
    
    N_s = len(seq) #N
    n_spins = len(seq[0]) #L
    n_h = alpha * n_spins #M
    
    
    theta = np.dot(w.T, seq.T)
    
    Oa = seq.T
    Ob = np.tanh(theta)
    Ow = seq.T.reshape((n_spins, 1, N_s)) * 
    np.tanh(theta.reshape(1, n_h, N_s))
    
    
    
    

$\theta$ M*N ottenuta da una M*L e una L*N quindi quando si somma B vettore di lunghezza M questo subirà un broadcasting lungo le righe
B: vettore M*1
W: L*M
seq: storico delle sequenze generate: N*L, N cambia a ogni iterazione
corrispondenza con codice kafischer:
n_h -> M
N_s -> N
n_spins -> L

In [14]:
#seq dovrà essere un'array di elementi di questo tipo
seq = np.ones(12)*1/2*np.sign(npr.random(12)-0.5)
alpha=4
N_s = len(seq)
n_spins = len(seq[0])
n_h = alpha * n_spins


#stdev of initialized weights
sigma = np.sqrt(4. / float(N_s+n_h))

#initialized weights
w = sigma * (2*np.random.rand(n_spins, n_h) - 1) #from -sigma to sigma 
a = sigma * (2*np.random.rand(n_spins) - 1)
b = np.zeros(n_h)



TypeError: object of type 'numpy.float64' has no len()